In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
from out_in_bb_backtest import setup_tsl,setup_stop_tp
import numpy as np
from backtesting import Strategy, Backtest
import warnings
warnings.filterwarnings("ignore")

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

ativo = 'EURUSD'
#ativo = 'WIN@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 14000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df_puro = df.rename(columns = {'real_volume':'vol'})
df2 = df_puro.copy()

In [5]:
time_period = 14
var_bb = 1.8
pontos = 100
rate_stop = 1
rate_tp = 2
spread = 0.00005
timeframe = 5

new_total = None
for time_period in [14,20]:
    for pontos in [0.0005,0.0008,0.0010,0.0015]:
        for var_bb in [1.8,2,2.2]:
            for rate_tp in [1,2,3,4]:
                df=df_puro.copy()

                bt = setup_stop_tp(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)
                #bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

                stats = bt.run()


                start = stats['Start'].date()
                end = stats['End'].date()

                n_trades = stats['# Trades']
                taxa_acerto = stats['Win Rate [%]']
                retorno = stats['Return [%]']

                acerto = n_trades * taxa_acerto/100
                erro = n_trades - acerto

                pontos_total = acerto * pontos * rate_tp - erro * pontos * rate_stop - spread * n_trades

                new = pd.DataFrame([[timeframe,pontos,rate_tp,time_period,var_bb,n_trades,taxa_acerto,pontos_total,retorno]])
                new.columns = ['Timeframe','pontos','Rate_tp','Time_period','var_bb','N_trades','Taxa_acerto','Pontos_total','Retorno']
                
                new_total = pd.concat([new_total,new])

                print(new_total.to_string())

   Timeframe  pontos  Rate_tp  Time_period  var_bb  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0005        1           14     1.8       556    51.978417       -0.0168  0.848174
   Timeframe  pontos  Rate_tp  Time_period  var_bb  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0005        1           14     1.8       556    51.978417      -0.01680  0.848174
0          5  0.0005        2           14     1.8       531    35.216573      -0.01155  0.111327
   Timeframe  pontos  Rate_tp  Time_period  var_bb  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0005        1           14     1.8       556    51.978417      -0.01680  0.848174
0          5  0.0005        2           14     1.8       531    35.216573      -0.01155  0.111327
0          5  0.0005        3           14     1.8       518    29.150579       0.01710  0.561017
   Timeframe  pontos  Rate_tp  Time_period  var_bb  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0005

In [6]:
new_total.sort_values(by = 'Pontos_total', ascending=False).iloc[:]

,Timeframe,pontos,Rate_tp,Time_period,var_bb,N_trades,Taxa_acerto,Pontos_total,Retorno
0,5,0.0015,4,14,1.8,344,47.383721,0.68930,6.150485
0,5,0.0015,4,14,2.0,329,44.072948,0.57755,1.129400
0,5,0.0015,4,14,2.2,273,44.322344,0.48435,3.507004
0,5,0.0015,4,20,1.8,235,44.680851,0.42325,4.517599
0,5,0.0015,3,14,2.0,333,46.546547,0.41385,4.345186
...,...,...,...,...,...,...,...,...,...
0,5,0.0005,1,14,2.2,455,53.626374,-0.00625,1.967509
0,5,0.0005,1,20,1.8,419,52.983294,-0.00845,1.115531
0,5,0.0008,1,20,1.8,395,51.645570,-0.00935,0.853673
0,5,0.0005,2,14,1.8,531,35.216573,-0.01155,0.111327


In [7]:
new_total.to_excel('resultado_eurusd_5M_ema_False_stop_tp.xlsx',index=False)

In [ ]:
time_period = 14
var_bb = 1.8
pontos = 100
rate_stop = 1
rate_tp = 2
spread = 5
timeframe = 5

bt = setup_stop_tp(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=True)

stats = bt.run()

#### Setup_Tsl

In [5]:
time_period = 14
var_bb = 1.8
pontos = 0.0010
rate_stop = 1
rate_tp = 2
spread = 0.00005
timeframe = 5

new_total = None
for time_period in [14,20,30]:
    for var_bb in [1.8,2,2.2]:
        df=df_puro.copy()
        bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=True)

        stats = bt.run()


        start = stats['Start'].date()
        end = stats['End'].date()

        n_trades = stats['# Trades']
        taxa_acerto = stats['Win Rate [%]']
        retorno = stats['Return [%]']

        acerto = n_trades * taxa_acerto/100
        erro = n_trades - acerto

        pontos_total = acerto * pontos * rate_tp - erro * pontos * rate_stop - spread * n_trades

        new = pd.DataFrame([[timeframe,pontos,time_period,var_bb,n_trades,taxa_acerto,retorno]])
        new.columns = ['Timeframe','pontos','Time_period','var_bb','N_trades','Taxa_acerto','Retorno']
        
        new_total = pd.concat([new_total,new])

        print(new_total.to_string())

   Timeframe  pontos  Time_period  var_bb  N_trades  Taxa_acerto   Retorno
0          5   0.001           14     1.8       205    32.682927 -1.534853
   Timeframe  pontos  Time_period  var_bb  N_trades  Taxa_acerto   Retorno
0          5   0.001           14     1.8       205    32.682927 -1.534853
0          5   0.001           14     2.0       182    40.659341  0.186249
   Timeframe  pontos  Time_period  var_bb  N_trades  Taxa_acerto   Retorno
0          5   0.001           14     1.8       205    32.682927 -1.534853
0          5   0.001           14     2.0       182    40.659341  0.186249
0          5   0.001           14     2.2       150    43.333333  0.581531
   Timeframe  pontos  Time_period  var_bb  N_trades  Taxa_acerto   Retorno
0          5   0.001           14     1.8       205    32.682927 -1.534853
0          5   0.001           14     2.0       182    40.659341  0.186249
0          5   0.001           14     2.2       150    43.333333  0.581531
0          5   0.001     

In [ ]:
new_total.to_excel('resultado_eurusd_5M_ema_True_tls.xlsx',index=False)

In [ ]:
bt = setup_tsl(df2,timeperiod=time_period,var_bb=var_bb)

stats = bt.run()

In [8]:
stats

Start                     2022-11-07 16:25:00
End                       2023-01-13 23:50:00
Duration                     67 days 07:25:00
Exposure Time [%]                   55.942857
Equity Final [$]             104518376.352924
Equity Peak [$]              104533809.256144
Return [%]                           4.518376
Buy & Hold Return [%]                8.402134
Return (Ann.) [%]                   25.391133
Volatility (Ann.) [%]                 7.47859
Sharpe Ratio                         3.395177
Sortino Ratio                        9.089393
Calmar Ratio                        14.134977
Max. Drawdown [%]                   -1.796333
Avg. Drawdown [%]                   -0.149229
Max. Drawdown Duration       23 days 14:45:00
Avg. Drawdown Duration        0 days 18:21:00
# Trades                                 1047
Win Rate [%]                        35.052531
Best Trade [%]                        0.54469
Worst Trade [%]                     -0.086637
Avg. Trade [%]                    

In [ ]:
bt.plot()

In [ ]:
print(stats['_trades'])